In [54]:

import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import pandas as pd 
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import scipy.io as scio

# Regularized_loss와 R_MLR 함수 정의
def Regularized_loss(model, n, y_pred, y_true, p=4, lam=0.01):
    classification_loss = -torch.mean(y_true * torch.log_softmax(y_pred, dim=1))
    RG_loss = 1/n * torch.norm(model.weight.unsqueeze(1) - model.weight.unsqueeze(0), p=2, dim=2).pow(p).sum()
    loss = classification_loss + lam * RG_loss
    return loss

def R_MLR(para):
    path = f'./dataset/{para["data"]}.mat'
    X = scio.loadmat(path)['X']
    y = scio.loadmat(path)['Y'].squeeze()
    print(X.shape, y.shape)

    n, d = X.shape[0], X.shape[1]
    num_class = len(np.unique(y))

    if para["If_scale"]:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

    y = y - 1

    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)

    X_train, X_test, y_train, y_test = \
        train_test_split(X_tensor, y_tensor, test_size=para["test_size"], random_state=para["state"])

    y_train = torch.nn.functional.one_hot(torch.tensor(y_train))
    y_test = torch.nn.functional.one_hot(torch.tensor(y_test))

    # 모델 및 옵티마이저 정의
    model = torch.nn.Linear(d, num_class)
    optimizer = torch.optim.Adam(model.parameters(), lr=para["lr"], weight_decay=para["weight_decay"])

    # 손실 값과 정확도 저장용 리스트 초기화
    loss_list = []
    test_acc_list = []

    for epoch in range(para["num_epoch"]):
        y_pred = model(X_train)
        loss = Regularized_loss(model, n, y_pred, y_train, para["p"], para["lam"])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            # print(f"Epoch [{epoch+1}/{para['num_epoch']}], Loss: {loss.item():.4f}")

            with torch.no_grad():
                y_pred = model(X_test)
                correct = (torch.argmax(y_pred, dim=1) == torch.argmax(y_test, dim=1)).sum().item()
                test_acc = correct / len(X_test)
                # print(f"Test Accuracy: {test_acc:.4f}")

            loss_list.append(loss.item())
            test_acc_list.append(test_acc)

    print(f"Final Test Accuracy: {test_acc:.4f}")
    return test_acc_list[-1]
    
# 데이터 별 파라미터 설정 함수
def set_parameters(data_name, para):
    para["data"] = data_name
    if data_name == 'HHAR':
        para.update({"lr": 0.02, "lam": 0.0005, "p": 4, "If_scale": True})
    elif data_name == 'Cornell':
        para.update({"lr": 0.1, "lam": 0.005, "p": 1, "If_scale": False})
    elif data_name == 'USPS':
        para.update({"lr": 0.01, "lam": 0.001, "p": 4, "If_scale": True})
    elif data_name == 'ISOLET':
        para.update({"lr": 0.001, "lam": 0.001, "p": 4, "If_scale": False})
    elif data_name == 'ORL':
        para.update({"lr": 0.01, "lam": 0.01, "p": 6, "If_scale": True})
    elif data_name == 'Dermatology':
        para.update({"lr": 0.01, "lam": 0.1, "p": 6, "If_scale": False})
    elif data_name == 'Vehicle':
        para.update({"lr": 0.05, "lam": 0.0001, "p": 4, "If_scale": True})
    elif data_name == 'Glass':
        para.update({"lr": 0.01, "lam": 0.0001, "p": 4, "If_scale": True})
    return para
default_para = {
    "data": "Vehicle",
    "num_epoch": 500,
    "lr": 0.05,
    "weight_decay": 0.0,
    "lam": 0.0001,
    "p": 4,
    "state": 42,
    "If_scale": True,
    "test_size": 0.2
}

In [55]:
data_list = ['Cornell', 'ISOLET','HHAR', 'USPS',  'ORL', 'Dermatology', 'Vehicle', 'Glass']

# 결과 저장 리스트
results = []

# 데이터별로 파라미터 업데이트 및 결과 저장
for data in data_list:
    para = default_para.copy()
    para = set_parameters(data, para)
    result = R_MLR(para)
    results.append({"data": data, "result": result})

    

(827, 4134) (827,)
Final Test Accuracy: 0.8675
(1560, 617) (1560,)
Final Test Accuracy: 0.9487
(10299, 561) (10299,)
Final Test Accuracy: 0.9801
(9298, 256) (9298,)
Final Test Accuracy: 0.9462
(400, 1024) (400,)
Final Test Accuracy: 0.9750
(366, 34) (366,)
Final Test Accuracy: 0.9865
(846, 18) (846,)
Final Test Accuracy: 0.8000
(214, 9) (214,)
Final Test Accuracy: 0.7442


In [ ]:
print(pd.DataFrame(results))

          data    result
0         HHAR  0.980583
1      Cornell  0.867470
2         USPS  0.947312
3       ISOLET  0.945513
4          ORL  0.975000
5  Dermatology  0.986486
6      Vehicle  0.800000
7        Glass  0.744186
